In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib
import yfinance as yfin
import MH4518.WY.Utils.constants as cs
import MH4518.WY.Utils.Dates as dates 
import MH4518.WY.Utils.payoff as pf
import MH4518.WY.Utils.VarianceRedFunc as vr 
import MH4518.WY.Utils.GBM as gbm 
import warnings
warnings.filterwarnings("ignore")
import MH4518.WY.Utils.ProductPrice as pp 


%reload_ext autoreload
%autoreload 2

In [2]:
def getdata():
    data = yfin.download(['LONN.SW', 'SIKA.SW'], period= 'max')['Adj Close']
    data.index = data.index.tz_localize(None)
    data = data.loc[cs.initial_fixing_date: cs.final_fixing_date]
    return data 

data = getdata()
data

[*********************100%***********************]  2 of 2 completed


Ticker,LONN.SW,SIKA.SW
Date,,
2023-04-27,542.120483,237.429871
2023-04-28,546.263367,242.368103
2023-05-02,545.868896,239.800232
2023-05-03,574.079590,241.479218
2023-05-04,572.895935,242.071823
...,...,...
2024-07-24,522.599976,262.000000
2024-07-25,559.799988,258.500000
2024-07-26,581.000000,262.000000


In [28]:
data["LONN.SW"].mean()

np.float64(465.3207055586803)

In [3]:
params_product = {
    'Denomination': 1000,
    'Coupon_Rate': (0.08 / 4) ,  # Quarterly coupon payments
}

# Control Variate 

In [31]:
sim_t = gbm.multi_asset_gbm_n_sims(plot= False, plotasset= False, nsims=10, data = data, fdos= cs.final_fixing_date)
sim_t.columns

Length of date_list: 1
Number of time steps (num_steps): 1


MultiIndex([('LONN.SW',  'sim1'),
            ('LONN.SW',  'sim2'),
            ('LONN.SW',  'sim3'),
            ('LONN.SW',  'sim4'),
            ('LONN.SW',  'sim5'),
            ('LONN.SW',  'sim6'),
            ('LONN.SW',  'sim7'),
            ('LONN.SW',  'sim8'),
            ('LONN.SW',  'sim9'),
            ('LONN.SW', 'sim10'),
            ('SIKA.SW',  'sim1'),
            ('SIKA.SW',  'sim2'),
            ('SIKA.SW',  'sim3'),
            ('SIKA.SW',  'sim4'),
            ('SIKA.SW',  'sim5'),
            ('SIKA.SW',  'sim6'),
            ('SIKA.SW',  'sim7'),
            ('SIKA.SW',  'sim8'),
            ('SIKA.SW',  'sim9'),
            ('SIKA.SW', 'sim10')],
           names=['Asset', 'Simulation'])

In [32]:
# Extract asset paths
lonza_path = sim_t['LONN.SW']
sika_path = sim_t['SIKA.SW']

In [33]:
sika_path

Simulation,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,sim10
2024-07-30,269.100006,269.100006,269.100006,269.100006,269.100006,269.100006,269.100006,269.100006,269.100006,269.100006


In [6]:
payoff_gbm_t = pf.payoff(paths1=lonza_path, paths2=sika_path, params=params_product, fdos = cs.initial_fixing_date)

In [26]:
def cv2(payoff_gbm, data: pd.DataFrame, fdos, original_sika):
    '''
    Takes in simulated payoff_gbm.
    E_Y is the mean of Lonza for a new set of randomness 
    
    Params:
        lonza_path: dataframe of lonza paths on the cs.initial_fixing_date for n sims
    
    '''
    params_product = {
    'Denomination': 1000,
    'Coupon_Rate': (0.08 / 4) ,  # Quarterly coupon payments
}


    # computing beta with initial set of GBM simulations 
    terminal_original_sika = original_sika.iloc[-1].values
    var_X = np.var(payoff_gbm, ddof = 1)
    var_Y = np.var(terminal_original_sika, ddof = 1)
    print('Var_Y:', var_Y)
    cov_matrix = np.cov(payoff_gbm, terminal_original_sika, ddof=1)
    cov_XY = cov_matrix[0,1]

    corr_XY =   cov_XY / np.sqrt(var_X * var_Y)  
    print(f"Correlation between X and Y: {corr_XY:.4f}") 
    beta = cov_XY / var_Y
    print(f"Beta (β) Coefficient: {beta}")
        # Compute control variate estimator

    # compute mean_X and mean_Y with new set of randomness
    sim_extra = gbm.multi_asset_gbm_n_sims(plot= False, plotasset= False, nsims=cs.n_sims, data=data, fdos = fdos) #new lonza set of randomness 
    sika_path_new = sim_extra['LONN.SW']
    sika_path_new_terminal = sika_path_new.iloc[-1].values
    lonza_path_new = sim_extra['SIKA.SW']
    lonza_path_new_terminal = lonza_path_new.iloc[-1].values
    payoff_extra = pf.payoff(lonza_path_new, sika_path_new, params_product, fdos) # new set of X
    mean_X = np.mean(payoff_extra)
    mean_Y = np.mean(lonza_path_new_terminal)
    E_Y = data["LONN.SW"].mean()
    theta_CV = mean_X + beta * (mean_Y - E_Y)
    print("Correction:", mean_Y - E_Y)

    var_theta_CV = var_X - (cov_XY ** 2) / var_Y
    variance_reduction = (var_X - var_theta_CV) / var_X * 100
    print(f"Variance Reduction Achieved: {variance_reduction:.2f}%")
    print("CV Estimate Payoff :", theta_CV)
    return theta_CV

In [29]:
pp.product_price()

date
2023-04-28    974.0
2023-05-02    972.7
2023-05-03    980.9
2023-05-04    985.1
2023-05-05    988.2
              ...  
2024-07-24    940.1
2024-07-25    985.6
2024-07-26    991.5
2024-07-29    991.1
2024-07-30    990.9
Name: value, Length: 309, dtype: float64

In [34]:
pp.product_price()


date
2023-04-28    974.0
2023-05-02    972.7
2023-05-03    980.9
2023-05-04    985.1
2023-05-05    988.2
              ...  
2024-07-24    940.1
2024-07-25    985.6
2024-07-26    991.5
2024-07-29    991.1
2024-07-30    990.9
Name: value, Length: 309, dtype: float64